# Integrate SageMaker Model with Watson Open Scale

Contents
- Setup
- Binding machine learning engine
- Subscriptions
- Performance monitor, scoring and payload logging
- Quality monitor and feedback logging
- Fairness, Drift monitoring and explanations

<a id="setup"></a>
## 1. Setup

### Requirements installation

In [1]:
!pip install sagemaker --no-cache | tail -n 1
!pip install --upgrade ibm-watson-openscale --no-cache | tail -n 1
!pip install --upgrade boto3 --no-cache | tail -n 1
!pip install -U pandas==1.2.5 | tail -n 1

**Action:** Restart the kernel.

### Model creation using [Amazon SageMaker](https://aws.amazon.com/sagemaker/)

- Download this [notebook](https://github.ibm.com/ibm-aws/ibm-aws-immersion-day-lab-1/blob/main/notebooks/RI-SageMaker-Deploy-Wstudio.ipynb) to create SageMaker model
- Run the notebook to train a SageMaker model and create deployment endpoint for online inference

### 1.2 Authentication

#### Generate IBM Cloud API key as per steps listed in the documentation

In [2]:
CLOUD_API_KEY = ''

#### Since we are using internal database associated with OpenScale instance, DB_CREDENTIALS are not needed.

In [3]:
DB_CREDENTIALS=None
#DB_CREDENTIALS = {"hostname":".cluster-cke9xrobcrxi.us-east-2.rds.amazonaws.com","username":"PostgresDBuser","password":"ImmersionDay!23","database":"immersiondayDB","port":"5432","ssl":False,"sslmode":"","certificate_base64":""}

In [4]:
SCHEMA_NAME = 'data_mart_for_ibm_aws'

In [5]:
IAM_URL="https://iam.ng.bluemix.net/oidc/token"
COS_RESOURCE_CRN=""
COS_API_KEY_ID = ""
COS_ENDPOINT = "https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints" # Current list avaiable at https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints

In [6]:
!wget "https://raw.githubusercontent.com/RK-Sharath/data/main/folder/Data-region-RI-SM.csv"

--2022-07-05 12:30:04--  https://raw.githubusercontent.com/RK-Sharath/data/main/folder/Data-region-RI-SM.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36731 (36K) [text/plain]
Saving to: ‘Data-region-RI-SM.csv.1’

Data-region-RI-SM.c 100%[===================>]  35.87K  --.-KB/s    in 0.01s   

2022-07-05 12:30:04 (3.14 MB/s) - ‘Data-region-RI-SM.csv.1’ saved [36731/36731]



In [7]:
BUCKET_NAME = "rksharathkumar-ibm-aws-05-22" 
training_data_file_name="Data-region-RI-SM.csv"

### Store training data in COS for OpenScale reference

In [8]:
import ibm_boto3
from ibm_botocore.client import Config, ClientError

cos_client = ibm_boto3.resource("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_RESOURCE_CRN,
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)

In [9]:
BUCKET_NAME

'rksharathkumar-ibm-aws-05-22'

In [10]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator,CloudPakForDataAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *

authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY)
wos_client = APIClient(authenticator=authenticator,service_url="https://aiopenscale.cloud.ibm.com")
wos_client.version

'3.0.19'

Create schema for data mart.

In [11]:
wos_client.data_marts.show()

None,None,True,active,2022-07-04 07:44:31.582000+00:00,0ea0e235-e74f-45b0-8dca-e3ac5e613620


In [12]:
data_marts = wos_client.data_marts.list().result.data_marts
if len(data_marts) == 0:
    if DB_CREDENTIALS is not None:
        if SCHEMA_NAME is None: 
            print("Please specify the SCHEMA_NAME and rerun the cell")

        print('Setting up external datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS SGM Data Mart",
                description="Data Mart created for WOS SGM",
                database_configuration=DatabaseConfigurationRequest(
                  database_type=DatabaseType.POSTGRESQL, # For DB2 use DatabaseType.DB2
                    credentials=PrimaryStorageCredentialsLong(
                        hostname=DB_CREDENTIALS['hostname'],
                        username=DB_CREDENTIALS['username'],
                        password=DB_CREDENTIALS['password'],
                        db=DB_CREDENTIALS['database'],
                        port=DB_CREDENTIALS['port'],
                        ssl=False,
                        certificate_base64=False
                    ),
                    location=LocationSchemaName(
                        schema_name= SCHEMA_NAME
                    )
                )
             ).result
    else:
        print('Setting up internal datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS SGM Data Mart",
                description="Data Mart created for WOS SGM", 
                internal_database = True).result
        
    data_mart_id = added_data_mart_result.metadata.id
    
else:
    data_mart_id=data_marts[0].metadata.id
    print('Using existing datamart {}'.format(data_mart_id))

Using existing datamart 0ea0e235-e74f-45b0-8dca-e3ac5e613620


In [13]:
wos_client.data_marts.get(data_mart_id).result.to_dict()

{'metadata': {'id': '0ea0e235-e74f-45b0-8dca-e3ac5e613620',
  'crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/f9ef21df684bc495941d94a200fb650b:0ea0e235-e74f-45b0-8dca-e3ac5e613620:data_mart:0ea0e235-e74f-45b0-8dca-e3ac5e613620',
  'url': '/v2/data_marts/0ea0e235-e74f-45b0-8dca-e3ac5e613620',
  'created_at': '2022-07-04T07:44:31.582000Z',
  'created_by': 'IBMid-50YWG7G0UB'},
 'entity': {'service_instance_crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/f9ef21df684bc495941d94a200fb650b:0ea0e235-e74f-45b0-8dca-e3ac5e613620::',
  'internal_database': True,
  'database_configuration': {'database_type': 'postgresql',
   'credentials': {'secret_id': 'bc17a850-0d03-44b9-a482-af8030b783f2'},
   'location': {'schema_name': '0ea0e235-e74f-45b0-8dca-e3ac5e613620'}},
  'status': {'state': 'active'}}}

<a id="binding"></a>
## 2. Bind machine learning engines

### Bind  `SageMaker` machine learning engine

Provide credentials using following fields:
- `access_key_id`
- `secret_access_key`
- `region`

In [14]:
SAGEMAKER_ENGINE_CREDENTIALS = {
                   'access_key_id': '', 
                   'secret_access_key': '', 
                   'region': 'us-east-2'}

In [15]:
SERVICE_PROVIDER_NAME = "AWS SGM Machine Learning"
SERVICE_PROVIDER_DESCRIPTION = "Added by AWS IBM Integration"

In [16]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

Deleted existing service_provider for WML instance: f3d0acca-6da9-4d07-8dfb-de6323189fc1


In [17]:
added_service_provider_result=wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description="AWS Service Provider",
        service_type=ServiceTypes.AMAZON_SAGEMAKER,
        credentials=SageMakerCredentials(
            access_key_id=SAGEMAKER_ENGINE_CREDENTIALS['access_key_id'],
            secret_access_key=SAGEMAKER_ENGINE_CREDENTIALS['secret_access_key'],
            region=SAGEMAKER_ENGINE_CREDENTIALS['region']
        ),
        background_mode=False
    ).result



service_provider_id = added_service_provider_result.metadata.id
print("Service Provider id ", service_provider_id)




 Waiting for end of adding service provider 21678a0a-fda7-4a58-9fa7-636be5f6a271 




active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------


Service Provider id  21678a0a-fda7-4a58-9fa7-636be5f6a271


In [18]:
wos_client.service_providers.show()

None,active,AWS SGM Machine Learning,amazon_sagemaker,2022-07-05 12:30:07.705000+00:00,21678a0a-fda7-4a58-9fa7-636be5f6a271
None,active,AWS ML Services,amazon_sagemaker,2022-07-05 11:50:18.620000+00:00,9095e229-2b90-42fc-8d17-0f97c1f9d7ed


In [19]:
asset_deployment_details = wos_client.service_providers.list_assets(data_mart_id=data_mart_id, service_provider_id=service_provider_id).result
asset_deployment_details

{'resources': [{'metadata': {'guid': '993fa28b93cba9b0d524f97a951a4b2c',
    'url': 'linear-learner-2022-06-27-07-53-34-124',
    'created_at': '2022-06-27T07:53:34.781Z',
    'modified_at': '2022-06-27T07:56:35.813Z'},
   'entity': {'name': 'linear-learner-2022-06-27-07-53-34-124',
    'deployment_rn': 'arn:aws:sagemaker:us-east-2:481118440516:endpoint/linear-learner-2022-06-27-07-53-34-124',
    'type': 'online',
    'scoring_endpoint': {'url': 'linear-learner-2022-06-27-07-53-34-124'},
    'asset': {'asset_id': '993fa28b93cba9b0d524f97a951a4b2c',
     'asset_rn': 'arn:aws:sagemaker:us-east-2:481118440516:model/linear-learner-2022-06-27-07-53-34-124',
     'url': 's3://sagemaker-us-east-2-481118440516/linear-learner-2022-06-27-07-49-21-672/output/model.tar.gz',
     'name': 'linear-learner-2022-06-27-07-53-34-124',
     'asset_type': 'model',
     'created_at': '2022-06-27T07:53:34.443Z'},
    'asset_properties': {'asset_revision': '1656316595813'}}}],
 'count': 1}

In [20]:
deployment_id='993fa28b93cba9b0d524f97a951a4b2c' 
for model_asset_details in asset_deployment_details['resources']:
    if model_asset_details['metadata']['guid']==deployment_id:
        break
model_asset_details

{'metadata': {'guid': '993fa28b93cba9b0d524f97a951a4b2c',
  'url': 'linear-learner-2022-06-27-07-53-34-124',
  'created_at': '2022-06-27T07:53:34.781Z',
  'modified_at': '2022-06-27T07:56:35.813Z'},
 'entity': {'name': 'linear-learner-2022-06-27-07-53-34-124',
  'deployment_rn': 'arn:aws:sagemaker:us-east-2:481118440516:endpoint/linear-learner-2022-06-27-07-53-34-124',
  'type': 'online',
  'scoring_endpoint': {'url': 'linear-learner-2022-06-27-07-53-34-124'},
  'asset': {'asset_id': '993fa28b93cba9b0d524f97a951a4b2c',
   'asset_rn': 'arn:aws:sagemaker:us-east-2:481118440516:model/linear-learner-2022-06-27-07-53-34-124',
   'url': 's3://sagemaker-us-east-2-481118440516/linear-learner-2022-06-27-07-49-21-672/output/model.tar.gz',
   'name': 'linear-learner-2022-06-27-07-53-34-124',
   'asset_type': 'model',
   'created_at': '2022-06-27T07:53:34.443Z'},
  'asset_properties': {'asset_revision': '1656316595813'}}}

<a id="subsciption"></a>
## 3. Subscriptions

### Add subscriptions

List available deployments.

**Note:** Depending on number of assets it may take some time.

In [21]:
aws_asset = Asset(
        asset_id=model_asset_details['entity']['asset']['asset_id'],
        name=model_asset_details['entity']['asset']['name'],
        url=model_asset_details['entity']['asset']['url'],
        asset_type=model_asset_details['entity']['asset']['asset_type'] if 'asset_type' in model_asset_details['entity']['asset'] else 'model',
        problem_type=ProblemType.BINARY_CLASSIFICATION,
        input_data_type=InputDataType.STRUCTURED,
    )

In [22]:
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import ScoringEndpointRequest
deployment_scoring_endpoint = model_asset_details['entity']['scoring_endpoint']
scoring_endpoint = ScoringEndpointRequest(url = model_asset_details['entity']['scoring_endpoint']['url'] )

In [23]:
deployment = AssetDeploymentRequest(
        deployment_id=model_asset_details['metadata']['guid'],
        url=model_asset_details['metadata']['url'],
        name=model_asset_details['entity']['name'],
        deployment_type=model_asset_details['entity']['type'],
        scoring_endpoint =  scoring_endpoint
    )

In [24]:
training_data_reference = TrainingDataReference(type='cos',
                                              location=COSTrainingDataReferenceLocation(bucket = BUCKET_NAME,
                                                                                        file_name = training_data_file_name),
                                              connection=COSTrainingDataReferenceConnection(
                                                                        resource_instance_id= COS_RESOURCE_CRN,
                                                                        url= COS_ENDPOINT,
                                                                        api_key= COS_API_KEY_ID,
                                                                        iam_url=IAM_URL)
                                               )

In [25]:
feature_columns = ['REGION','Total_cases']
categorical_columns = ['REGION']

In [26]:
asset_properties = AssetPropertiesRequest(
        label_column="Risk_Index",
        prediction_field='predicted_label',
        probability_fields=['score'],
        training_data_reference=training_data_reference,
        training_data_schema=None,
        input_data_schema=None,
        output_data_schema=None,
        feature_fields=feature_columns,
        categorical_fields=categorical_columns
    )

In [27]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=data_mart_id,
        service_provider_id=service_provider_id,
        asset=aws_asset,
        deployment=deployment,
        asset_properties=asset_properties,
        background_mode=False
).result
subscription_id = subscription_details.metadata.id
subscription_id




 Waiting for end of adding subscription f8ed9823-c577-457b-a164-2dafed26ff02 




active

-------------------------------------------
 Successfully finished adding subscription 
-------------------------------------------




'f8ed9823-c577-457b-a164-2dafed26ff02'

#### List subscriptions

In [28]:
wos_client.subscriptions.show()

993fa28b93cba9b0d524f97a951a4b2c,linear-learner-2022-06-27-07-53-34-124,0ea0e235-e74f-45b0-8dca-e3ac5e613620,993fa28b93cba9b0d524f97a951a4b2c,linear-learner-2022-06-27-07-53-34-124,21678a0a-fda7-4a58-9fa7-636be5f6a271,active,2022-07-05 12:30:14.874000+00:00,f8ed9823-c577-457b-a164-2dafed26ff02
993fa28b93cba9b0d524f97a951a4b2c,linear-learner-2022-06-27-07-53-34-124,0ea0e235-e74f-45b0-8dca-e3ac5e613620,993fa28b93cba9b0d524f97a951a4b2c,linear-learner-2022-06-27-07-53-34-124,9095e229-2b90-42fc-8d17-0f97c1f9d7ed,active,2022-07-05 11:51:32.662000+00:00,94e8acef-5eb6-493b-bfaf-09f4cf8f5da0


<a id="scoring"></a>
## 4. Performance metrics, scoring and payload logging

### Score the risk index model and measure response time

In [29]:
import requests
import time
import json
import boto3

In [30]:
subscription_details=wos_client.subscriptions.get(subscription_id).result.to_dict()
subscription_details

{'metadata': {'id': 'f8ed9823-c577-457b-a164-2dafed26ff02',
  'crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/f9ef21df684bc495941d94a200fb650b:0ea0e235-e74f-45b0-8dca-e3ac5e613620:subscription:f8ed9823-c577-457b-a164-2dafed26ff02',
  'url': '/v2/subscriptions/f8ed9823-c577-457b-a164-2dafed26ff02',
  'created_at': '2022-07-05T12:30:14.874000Z',
  'created_by': 'IBMid-50YWG7G0UB'},
 'entity': {'data_mart_id': '0ea0e235-e74f-45b0-8dca-e3ac5e613620',
  'service_provider_id': '21678a0a-fda7-4a58-9fa7-636be5f6a271',
  'asset': {'asset_id': '993fa28b93cba9b0d524f97a951a4b2c',
   'url': 's3://sagemaker-us-east-2-481118440516/linear-learner-2022-06-27-07-49-21-672/output/model.tar.gz',
   'name': 'linear-learner-2022-06-27-07-53-34-124',
   'asset_type': 'model',
   'problem_type': 'binary',
   'input_data_type': 'structured'},
  'asset_properties': {'training_data_reference': {'secret_id': 'cb7f0dfc-c3e4-4b87-a024-70c104c937f3'},
   'output_data_schema': {'type': 'struct',
    'fields': [

In [31]:
endpoint_name = subscription_details['entity']['deployment']['name']

payload = "0,100"

In [32]:
runtime = boto3.client('sagemaker-runtime',
                       region_name=SAGEMAKER_ENGINE_CREDENTIALS['region'],
                       aws_access_key_id=SAGEMAKER_ENGINE_CREDENTIALS['access_key_id'],
                       aws_secret_access_key=SAGEMAKER_ENGINE_CREDENTIALS['secret_access_key'])

start_time = time.time()
response = runtime.invoke_endpoint(EndpointName=endpoint_name, ContentType='text/csv', Body=payload)
response_time = int((time.time() - start_time)*1000)
result = json.loads(response['Body'].read().decode())

print(json.dumps(result, indent=2))

{
  "predictions": [
    {
      "score": [
        0.8418549299240112,
        0.023631207644939423,
        0.13451385498046875
      ],
      "predicted_label": 0
    }
  ]
}


### Store the request and response in payload logging table

#### Transform the model's input and output to the format compatible with OpenScale standard.

In [33]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id: ", payload_data_set_id)

Payload data set id:  d977140a-9d48-4110-95a5-08bb73740022


In [34]:
values = [float(s) for s in payload.split(',')]

request_data = {'fields': feature_columns, 
                'values': values}

response_data = {'fields': list(result['predictions'][0]),
                 'values': [list(x.values()) for x in result['predictions']]}

#### Store the payload using Python SDK

**Hint:** You can embed payload logging code into your custom deployment so it is logged automatically each time you score the model.

In [35]:
import uuid
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord

print("Performing explicit payload logging.....")
wos_client.data_sets.store_records(data_set_id=payload_data_set_id, background_mode=False,request_body=[PayloadRecord(
           scoring_id=str(uuid.uuid4()),
           request=request_data,
           response=response_data,
           response_time=460
)])
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))

Performing explicit payload logging.....



 Waiting for end of storing records with request id: e5fe8a88-42ec-41ab-b320-55bfbb2e6bac 




active

---------------------------------------
 Successfully finished storing records 
---------------------------------------


Number of records in the payload logging table: 1


In [36]:
wos_client.data_sets.show_records(data_set_id=payload_data_set_id)

None,526db926-a4aa-4571-9ea3-6f84fb164a50-1,0,2022-07-05T12:30:25.933Z,0.8418549299240112,"[0.8418549299240112, 0.023631207644939423, 0.13451385498046875]",0.0,993fa28b93cba9b0d524f97a951a4b2c,100.0


<a id="feedback"></a>
## 5. Feedback logging & quality (accuracy) monitoring

### Enable quality monitoring

You need to provide the monitoring `threshold` and `min_records` (minimal number of feedback records).

In [37]:
import time

time.sleep(10)
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 10
}
thresholds = [
                {
                    "metric_id": "area_under_roc",
                    "type": "lower_limit",
                    "value": .80
                }
            ]
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result




 Waiting for end of monitor instance creation 8f565216-8450-4764-a95a-3d5ce57db3b3 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [38]:
quality_monitor_instance_id = quality_monitor_details.metadata.id
quality_monitor_instance_id

'8f565216-8450-4764-a95a-3d5ce57db3b3'

### Feedback records logging

Feedback records are used to evaluate your model. The predicted values are compared to real values (feedback records).

You can check the schema of feedback table using below method.

In [39]:
feedback_dataset_id = None
feedback_dataset = wos_client.data_sets.list(type=DataSetTypes.FEEDBACK, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result
feedback_dataset_id = feedback_dataset.data_sets[0].metadata.id
if feedback_dataset_id is None:
    print("Feedback data set not found. Please check quality monitor status.")
feedback_dataset_id

'a4c66282-ccf0-483c-8a51-8f97033cdd8e'

The feedback records can be send to feedback table using below code.

In [40]:
import requests
import pandas as pd
import numpy as np
import time

time.sleep(10) #It gives enough time for dataset creation

data = pd.read_csv('https://raw.githubusercontent.com/RK-Sharath/data/main/folder/Data-region-RI-SM-Feedback.csv',header=0,dtype=np.float)
feedback_columns = data.columns.tolist()
feedback_records = data.values.tolist()

payload_scoring =  [{"fields": feedback_columns, "values": feedback_records}]
wos_client.data_sets.store_records(feedback_dataset_id, request_body=payload_scoring, background_mode=False)

/tmp/1000650000/ipykernel_420/1337157720.py:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data = pd.read_csv('https://raw.githubusercontent.com/RK-Sharath/data/main/folder/Data-region-RI-SM-Feedback.csv',header=0,dtype=np.float)





 Waiting for end of storing records with request id: 633598e2-81ef-491e-a64d-6f74b76d81d4 




active

---------------------------------------
 Successfully finished storing records 
---------------------------------------




In [41]:
wos_client.data_sets.print_records_schema(data_set_id=feedback_dataset_id)

REGION,double,True
Total_cases,double,True
Risk_Index,double,True
record_id,string,False
record_timestamp,timestamp,False
transaction_id,string,True
_original_prediction,integer,True
_original_probability,"{'containsNull': True, 'elementType': 'double', 'type': 'array'}",True
_debiased_prediction,integer,True
_debiased_probability,"{'containsNull': True, 'elementType': 'double', 'type': 'array'}",True


In [42]:
wos_client.data_sets.get_records_count(data_set_id=feedback_dataset_id)

12

In [43]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id, background_mode=False).result




 Waiting for end of monitoring run 5da76a46-b5bd-4fc3-8563-8dd96042d9fb 




running
finished

---------------------------
 Successfully finished run 
---------------------------




In [44]:
time.sleep(5)
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

2022-07-05 12:31:10.114000+00:00,true_positive_rate,5e9404c2-a60d-42c1-a8d0-993a41ac8e2a,0.6666666666666666,None,None,['model_type:original'],quality,8f565216-8450-4764-a95a-3d5ce57db3b3,5da76a46-b5bd-4fc3-8563-8dd96042d9fb,subscription,f8ed9823-c577-457b-a164-2dafed26ff02
2022-07-05 12:31:10.114000+00:00,area_under_roc,5e9404c2-a60d-42c1-a8d0-993a41ac8e2a,0.8333333333333333,0.8,None,['model_type:original'],quality,8f565216-8450-4764-a95a-3d5ce57db3b3,5da76a46-b5bd-4fc3-8563-8dd96042d9fb,subscription,f8ed9823-c577-457b-a164-2dafed26ff02
2022-07-05 12:31:10.114000+00:00,precision,5e9404c2-a60d-42c1-a8d0-993a41ac8e2a,1.0,None,None,['model_type:original'],quality,8f565216-8450-4764-a95a-3d5ce57db3b3,5da76a46-b5bd-4fc3-8563-8dd96042d9fb,subscription,f8ed9823-c577-457b-a164-2dafed26ff02
2022-07-05 12:31:10.114000+00:00,f1_measure,5e9404c2-a60d-42c1-a8d0-993a41ac8e2a,0.8,None,None,['model_type:original'],quality,8f565216-8450-4764-a95a-3d5ce57db3b3,5da76a46-b5bd-4fc3-8563-8dd96042d9fb,subscription,f8ed9823-c577-457b-a164-2dafed26ff02
2022-07-05 12:31:10.114000+00:00,accuracy,5e9404c2-a60d-42c1-a8d0-993a41ac8e2a,0.9166666666666666,None,None,['model_type:original'],quality,8f565216-8450-4764-a95a-3d5ce57db3b3,5da76a46-b5bd-4fc3-8563-8dd96042d9fb,subscription,f8ed9823-c577-457b-a164-2dafed26ff02
2022-07-05 12:31:10.114000+00:00,log_loss,5e9404c2-a60d-42c1-a8d0-993a41ac8e2a,0.7617815096623223,None,None,['model_type:original'],quality,8f565216-8450-4764-a95a-3d5ce57db3b3,5da76a46-b5bd-4fc3-8563-8dd96042d9fb,subscription,f8ed9823-c577-457b-a164-2dafed26ff02
2022-07-05 12:31:10.114000+00:00,false_positive_rate,5e9404c2-a60d-42c1-a8d0-993a41ac8e2a,0.0,None,None,['model_type:original'],quality,8f565216-8450-4764-a95a-3d5ce57db3b3,5da76a46-b5bd-4fc3-8563-8dd96042d9fb,subscription,f8ed9823-c577-457b-a164-2dafed26ff02
2022-07-05 12:31:10.114000+00:00,area_under_pr,5e9404c2-a60d-42c1-a8d0-993a41ac8e2a,0.875,None,None,['model_type:original'],quality,8f565216-8450-4764-a95a-3d5ce57db3b3,5da76a46-b5bd-4fc3-8563-8dd96042d9fb,subscription,f8ed9823-c577-457b-a164-2dafed26ff02
2022-07-05 12:31:10.114000+00:00,recall,5e9404c2-a60d-42c1-a8d0-993a41ac8e2a,0.6666666666666666,None,None,['model_type:original'],quality,8f565216-8450-4764-a95a-3d5ce57db3b3,5da76a46-b5bd-4fc3-8563-8dd96042d9fb,subscription,f8ed9823-c577-457b-a164-2dafed26ff02


<a id="datamart"></a>
## 6. Get the logged data

### Payload logging

#### Print schema of payload_logging table

In [45]:
wos_client.data_sets.print_records_schema(data_set_id=payload_data_set_id)

scoring_id,string,False
scoring_timestamp,timestamp,False
deployment_id,string,False
asset_revision,string,True
REGION,double,True
Total_cases,double,True
score,"{'containsNull': True, 'elementType': 'double', 'type': 'array'}",True
predicted_label,integer,True
prediction_probability,double,True


<a id="fairness_and_explainability"></a>
## 7. Fairness, Drift monitoring and explanations

### Get payload data

In [46]:
#scoring_data_filename='Data-region-RI-SM-Scoring.csv'
scoring_data_filename_json='Data-region-RI-SM-Scoring.json'

In [47]:
!rm Data-region-RI-SM-Scoring.json
!rm Data-region-RI-SM-Scoring.csv
!wget "https://raw.githubusercontent.com/RK-Sharath/data/main/folder/Data-region-RI-SM-Scoring.json"
!wget "https://raw.githubusercontent.com/RK-Sharath/data/main/folder/Data-region-RI-SM-Scoring.csv"

--2022-07-05 12:31:27--  https://raw.githubusercontent.com/RK-Sharath/data/main/folder/Data-region-RI-SM-Scoring.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 106 [text/plain]
Saving to: ‘Data-region-RI-SM-Scoring.json’

Data-region-RI-SM-S 100%[===================>]     106  --.-KB/s    in 0s      

2022-07-05 12:31:28 (7.86 MB/s) - ‘Data-region-RI-SM-Scoring.json’ saved [106/106]

--2022-07-05 12:31:28--  https://raw.githubusercontent.com/RK-Sharath/data/main/folder/Data-region-RI-SM-Scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting respo

In [48]:
data=pd.read_csv('https://raw.githubusercontent.com/RK-Sharath/data/main/folder/Data-region-RI-SM-Scoring.csv')

In [49]:
import io
from io import StringIO
csv_file = io.StringIO()
# by default sagemaker expects comma seperated
data.to_csv(csv_file, sep=",", header=False, index=False)
scoring_data_filename = csv_file.getvalue()

In [50]:
sm_runtime = boto3.client('sagemaker-runtime',
                       region_name=SAGEMAKER_ENGINE_CREDENTIALS['region'],
                       aws_access_key_id=SAGEMAKER_ENGINE_CREDENTIALS['access_key_id'],
                       aws_secret_access_key=SAGEMAKER_ENGINE_CREDENTIALS['secret_access_key'])


scoring_response = sm_runtime.invoke_endpoint(EndpointName = endpoint_name,
                                                  ContentType = 'text/csv',
                                                  Body = scoring_data_filename)
    
result = json.loads(scoring_response['Body'].read().decode())
print(json.dumps(result, indent=2))

{
  "predictions": [
    {
      "score": [
        0.02799299918115139,
        0.7118191719055176,
        0.2601878345012665
      ],
      "predicted_label": 1
    },
    {
      "score": [
        0.8067101836204529,
        0.02732161432504654,
        0.16596820950508118
      ],
      "predicted_label": 0
    },
    {
      "score": [
        0.010750409215688705,
        0.19208674132823944,
        0.7971628308296204
      ],
      "predicted_label": 2
    },
    {
      "score": [
        0.03177503123879433,
        0.7168406248092651,
        0.25138434767723083
      ],
      "predicted_label": 1
    },
    {
      "score": [
        0.7829863429069519,
        0.029666010290384293,
        0.18734769523143768
      ],
      "predicted_label": 0
    },
    {
      "score": [
        0.060710735619068146,
        0.26483166217803955,
        0.6744576096534729
      ],
      "predicted_label": 2
    },
    {
      "score": [
        0.041998133063316345,
        0.72588139

In [51]:
f = open(scoring_data_filename_json,"r")
payload_values = json.load(f)
request_data = {'fields': feature_columns, 
                'values': payload_values}

response_data = {'fields': list(result['predictions'][0]),
                 'values': [list(x.values()) for x in result['predictions']]}


In [52]:
import uuid
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord

print("Performing explicit payload logging.....")
wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
           scoring_id=str(uuid.uuid4()),
           request=request_data,
           response=response_data,
           response_time=460
)])
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))

Performing explicit payload logging.....
Number of records in the payload logging table: 13


In [53]:
wos_client.data_sets.show_records(payload_data_set_id)

None,c6ceaa35-659d-4b47-bcd8-2337846edcbf-1,1.0,2022-07-05T12:31:28.537Z,0.7118191719055176,"[0.02799299918115139, 0.7118191719055176, 0.2601878345012665]",1.0,993fa28b93cba9b0d524f97a951a4b2c,10575.0
None,c6ceaa35-659d-4b47-bcd8-2337846edcbf-10,0.0,2022-07-05T12:31:28.537Z,0.7493403553962708,"[0.7493403553962708, 0.03281030058860779, 0.21784931421279907]",2.0,993fa28b93cba9b0d524f97a951a4b2c,200.0
None,c6ceaa35-659d-4b47-bcd8-2337846edcbf-11,0.0,2022-07-05T12:31:28.537Z,0.826548159122467,"[0.826548159122467, 0.025272492319345474, 0.1481793373823166]",0.0,993fa28b93cba9b0d524f97a951a4b2c,1400.0
None,c6ceaa35-659d-4b47-bcd8-2337846edcbf-12,nan,2022-07-05T12:31:28.537Z,nan,[],0.0,993fa28b93cba9b0d524f97a951a4b2c,350.0
None,c6ceaa35-659d-4b47-bcd8-2337846edcbf-2,0.0,2022-07-05T12:31:28.537Z,0.8067101836204529,"[0.8067101836204529, 0.02732161432504654, 0.16596820950508118]",2.0,993fa28b93cba9b0d524f97a951a4b2c,2100.0
None,c6ceaa35-659d-4b47-bcd8-2337846edcbf-3,2.0,2022-07-05T12:31:28.537Z,0.7971628308296204,"[0.010750409215688705, 0.19208674132823944, 0.7971628308296204]",0.0,993fa28b93cba9b0d524f97a951a4b2c,649.0
None,c6ceaa35-659d-4b47-bcd8-2337846edcbf-4,1.0,2022-07-05T12:31:28.537Z,0.7168406248092651,"[0.03177503123879433, 0.7168406248092651, 0.25138434767723083]",1.0,993fa28b93cba9b0d524f97a951a4b2c,10023.0
None,c6ceaa35-659d-4b47-bcd8-2337846edcbf-5,0.0,2022-07-05T12:31:28.537Z,0.7829863429069519,"[0.7829863429069519, 0.029666010290384293, 0.18734769523143768]",2.0,993fa28b93cba9b0d524f97a951a4b2c,1750.0
None,c6ceaa35-659d-4b47-bcd8-2337846edcbf-6,2.0,2022-07-05T12:31:28.537Z,0.6744576096534729,"[0.060710735619068146, 0.26483166217803955, 0.6744576096534729]",0.0,993fa28b93cba9b0d524f97a951a4b2c,977.0
None,c6ceaa35-659d-4b47-bcd8-2337846edcbf-7,1.0,2022-07-05T12:31:28.537Z,0.7258813977241516,"[0.041998133063316345, 0.7258813977241516, 0.23212049901485443]",1.0,993fa28b93cba9b0d524f97a951a4b2c,5900.0


### Enable and run fairness monitoring

In [54]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "features": [
        {"feature": "REGION",
         "majority": [[1,1]],
         "minority": [[0,0]],
         "threshold": 0.95
         }
    ],
    "favourable_class": [0],
    "unfavourable_class": [2],
    "min_records": 10
}

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters).result
fairness_monitor_instance_id =fairness_monitor_details.metadata.id
fairness_monitor_instance_id




 Waiting for end of monitor instance creation 86fce91e-491f-4c66-8c54-aabb8cfbfd2e 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




'86fce91e-491f-4c66-8c54-aabb8cfbfd2e'

### Run fairness monitor

In [55]:
time.sleep(20)
#Note: When you create fairness monitor, initial run is also created
wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

2022-07-05 12:31:40.288554+00:00,fairness_value,3988d1ab-9a2a-4082-9267-11ab91ebff44,100.0,80.0,None,"['feature:REGION', 'fairness_metric_type:fairness', 'feature_value:0-0']",fairness,86fce91e-491f-4c66-8c54-aabb8cfbfd2e,ffba8785-73da-4f94-a721-64855855886d,subscription,f8ed9823-c577-457b-a164-2dafed26ff02
2022-07-05 12:31:40.288554+00:00,fairness_value,9b382303-5be6-40aa-ab67-f5b6af709f06,100.0,80.0,None,"['feature:REGION', 'fairness_metric_type:debiased_fairness', 'feature_value:0-0']",fairness,86fce91e-491f-4c66-8c54-aabb8cfbfd2e,ffba8785-73da-4f94-a721-64855855886d,subscription,f8ed9823-c577-457b-a164-2dafed26ff02


### Enable and run Drift monitoring

#### Drift requires a trained model to be uploaded manually for AWS. Please check the documentation for more details.

### Enable Explainability and run explanation on sample record

In [56]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result




 Waiting for end of monitor instance creation 47917633-3a2b-4150-a4fb-47539c3e7947 




error

-----------------------------------------------------
 Monitor instance creation failed with status: error 
-----------------------------------------------------


Reason: ['code: AIQCS0002E, message: Action `Enable Monitor` has failed with status code 400', 'code: AIQES2032E, message: Binary classification model can not have more than two class labels']


Getting a `transaction_id` to run explanation on

In [57]:
explainability_monitor_id = explainability_details.metadata.id
explainability_monitor_id

'47917633-3a2b-4150-a4fb-47539c3e7947'

In [58]:
wos_client.data_sets.show_records(data_set_id=payload_data_set_id)

None,"[0.02799299918115139, 0.7118191719055176, 0.2601878345012665]",c6ceaa35-659d-4b47-bcd8-2337846edcbf-1,1.0,2022-07-05T12:31:28.537Z,0.7118191719055176,"[0.02799299918115139, 0.7118191719055176, 0.2601878345012665]",1.0,993fa28b93cba9b0d524f97a951a4b2c,10575.0,1.0
None,"[0.7493403553962708, 0.03281030058860779, 0.21784931421279907]",c6ceaa35-659d-4b47-bcd8-2337846edcbf-10,0.0,2022-07-05T12:31:28.537Z,0.7493403553962708,"[0.7493403553962708, 0.03281030058860779, 0.21784931421279907]",2.0,993fa28b93cba9b0d524f97a951a4b2c,200.0,0.0
None,"[0.826548159122467, 0.025272492319345474, 0.1481793373823166]",c6ceaa35-659d-4b47-bcd8-2337846edcbf-11,0.0,2022-07-05T12:31:28.537Z,0.826548159122467,"[0.826548159122467, 0.025272492319345474, 0.1481793373823166]",0.0,993fa28b93cba9b0d524f97a951a4b2c,1400.0,0.0
None,None,c6ceaa35-659d-4b47-bcd8-2337846edcbf-12,nan,2022-07-05T12:31:28.537Z,nan,[],0.0,993fa28b93cba9b0d524f97a951a4b2c,350.0,nan
None,"[0.8067101836204529, 0.02732161432504654, 0.16596820950508118]",c6ceaa35-659d-4b47-bcd8-2337846edcbf-2,0.0,2022-07-05T12:31:28.537Z,0.8067101836204529,"[0.8067101836204529, 0.02732161432504654, 0.16596820950508118]",2.0,993fa28b93cba9b0d524f97a951a4b2c,2100.0,0.0
None,"[0.010750409215688705, 0.19208674132823944, 0.7971628308296204]",c6ceaa35-659d-4b47-bcd8-2337846edcbf-3,2.0,2022-07-05T12:31:28.537Z,0.7971628308296204,"[0.010750409215688705, 0.19208674132823944, 0.7971628308296204]",0.0,993fa28b93cba9b0d524f97a951a4b2c,649.0,2.0
None,"[0.03177503123879433, 0.7168406248092651, 0.25138434767723083]",c6ceaa35-659d-4b47-bcd8-2337846edcbf-4,1.0,2022-07-05T12:31:28.537Z,0.7168406248092651,"[0.03177503123879433, 0.7168406248092651, 0.25138434767723083]",1.0,993fa28b93cba9b0d524f97a951a4b2c,10023.0,1.0
None,"[0.7829863429069519, 0.029666010290384293, 0.18734769523143768]",c6ceaa35-659d-4b47-bcd8-2337846edcbf-5,0.0,2022-07-05T12:31:28.537Z,0.7829863429069519,"[0.7829863429069519, 0.029666010290384293, 0.18734769523143768]",2.0,993fa28b93cba9b0d524f97a951a4b2c,1750.0,0.0
None,"[0.060710735619068146, 0.26483166217803955, 0.6744576096534729]",c6ceaa35-659d-4b47-bcd8-2337846edcbf-6,2.0,2022-07-05T12:31:28.537Z,0.6744576096534729,"[0.060710735619068146, 0.26483166217803955, 0.6744576096534729]",0.0,993fa28b93cba9b0d524f97a951a4b2c,977.0,2.0
None,"[0.041998133063316345, 0.7258813977241516, 0.23212049901485443]",c6ceaa35-659d-4b47-bcd8-2337846edcbf-7,1.0,2022-07-05T12:31:28.537Z,0.7258813977241516,"[0.041998133063316345, 0.7258813977241516, 0.23212049901485443]",1.0,993fa28b93cba9b0d524f97a951a4b2c,5900.0,1.0


In [59]:
payload_data = wos_client.data_sets.get_list_of_records(limit=5,data_set_id=payload_data_set_id,output_type='pandas').result
scoring_ids=payload_data['scoring_id'].tolist()
print("Running explanations on scoring IDs: {}".format(scoring_ids))
explanation_types = ["lime", "contrastive"]
result = wos_client.monitor_instances.explanation_tasks(scoring_ids=scoring_ids, explanation_types=explanation_types).result
print(result)
explanation_task_id=result.to_dict()['metadata']['explanation_task_ids'][0]
explanation_task_id

Running explanations on scoring IDs: ['c6ceaa35-659d-4b47-bcd8-2337846edcbf-1', 'c6ceaa35-659d-4b47-bcd8-2337846edcbf-10', 'c6ceaa35-659d-4b47-bcd8-2337846edcbf-11', 'c6ceaa35-659d-4b47-bcd8-2337846edcbf-12', 'c6ceaa35-659d-4b47-bcd8-2337846edcbf-2']
{
  "metadata": {
    "explanation_task_ids": [
      "48aa1ad7-69b3-4d46-8574-afcc5dab6335",
      "f4ae0825-4c13-4065-b02f-8451758768e4",
      "d94c1c18-151f-4f2c-8fe8-c227d2c24e8b",
      "8e0de448-d196-4174-b51e-bdf48aac7cee",
      "4039d2bc-f038-45d3-b1dd-dfe121578b52"
    ],
    "created_by": "IBMid-50YWG7G0UB",
    "created_at": "2022-07-05T12:32:08.127180Z"
  }
}


'48aa1ad7-69b3-4d46-8574-afcc5dab6335'

In [60]:
wos_client.monitor_instances.get_explanation_tasks(explanation_task_id=explanation_task_id).result.to_dict()

{'metadata': {'explanation_task_id': '48aa1ad7-69b3-4d46-8574-afcc5dab6335',
  'created_by': 'IBMid-50YWG7G0UB',
  'created_at': '2022-07-05T12:32:08.127180Z',
  'updated_at': '2022-07-05T12:32:09.580889Z'},
 'entity': {'status': {'state': 'in_progress'},
  'scoring_id': 'c6ceaa35-659d-4b47-bcd8-2337846edcbf-1'}}

You can now view the [OpenScale Dashboard](https://aiopenscale.cloud.ibm.com/). Click on the tile for the Risk Index AWS model to see fairness, accuracy, and performance monitors. 

### In this Notebook, we have learnt how to configure the metrics for monitoring the SageMaker model using Watson OpenScale. Fairness & Quality metrics have been programmatically configuered using this notebook. Drift model has to be configuered manually using the steps listed in the documentation. Explainability metric works for Binary Classification only and Risk Index prediction is multiclass classification (three classes).

---